In [4]:
!pip3 install --user ipykernel
!python3 -m ipykernel install --user --name=aue8930
!jupyter kernelspec list
#!jupyter kernelspec uninstall aue8930

     |████████████████████████████████| 121 kB 4.6 MB/s eta 0:00:01
     |████████████████████████████████| 130 kB 19.9 MB/s eta 0:00:01
     |████████████████████████████████| 427 kB 77.3 MB/s eta 0:00:01
     |████████████████████████████████| 783 kB 107.8 MB/s eta 0:00:01
     |████████████████████████████████| 75 kB 9.2 MB/s  eta 0:00:01
     |████████████████████████████████| 1.1 MB 109.0 MB/s eta 0:00:01
     |████████████████████████████████| 59 kB 13.9 MB/s eta 0:00:01
     |████████████████████████████████| 1.4 MB 102.7 MB/s eta 0:00:01
     |████████████████████████████████| 386 kB 101.1 MB/s eta 0:00:01
     |████████████████████████████████| 109 kB 106.1 MB/s eta 0:00:01
     |████████████████████████████████| 247 kB 110.9 MB/s eta 0:00:01
     |████████████████████████████████| 86 kB 13.8 MB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 64.7 MB/s eta 0:00:01
Installed kernelspec aue8930 in /home/csamak/.local/share/jupyter/kernels/aue8930
Available kernels:


In [1]:
import models
import os
import torch
from torch import optim, nn
from torchvision import transforms, datasets
import torchvision
import common

In [2]:
#DIRECTORY SETTINGS
os.chdir("..")#Go up two directories
SAVE_DIR = 'models'
MODEL_SAVE_PATH = os.path.join(SAVE_DIR, 'base.pt')


#HYPERPARAMETERS
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

EPOCHS=100
BATCH_SIZE = 32
criterion = nn.CrossEntropyLoss()
ADAM_OPTIMISER=True
LEARNING_RATE=0.001

In [3]:
train_transforms = transforms.Compose([# Data Transforms
                           transforms.Resize(256),#Resize
                           transforms.RandomHorizontalFlip(30),#Flip
                           transforms.RandomRotation(10),#Roatate
                           transforms.RandomCrop(256),#Crop
                           transforms.ToTensor(),#Convert to Tensor
                           transforms.Normalize((0.49139968, 0.48215827, 0.44653124), (0.24703233, 0.24348505, 0.26158768))#Normalize
                       ])

test_transforms = transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.49139968, 0.48215827, 0.44653124), (0.24703233, 0.24348505, 0.26158768))
                       ])

In [ ]:
train_data = torchvision.datasets.CIFAR10(root='data', train=True, download=True, transform=train_transforms)#Use CIFAR10 to train
train_data, valid_data = torch.utils.data.random_split(train_data, [int(len(train_data)*0.9), len(train_data) - int(len(train_data)*0.9)])
test_data = torchvision.datasets.CIFAR10(root='data', train=False, download=True, transform=test_transforms)


print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')


train_iterator = torch.utils.data.DataLoader(train_data, shuffle=True, batch_size=BATCH_SIZE)
valid_iterator = torch.utils.data.DataLoader(valid_data, batch_size=BATCH_SIZE)
test_iterator = torch.utils.data.DataLoader(test_data, batch_size=BATCH_SIZE)

0.0%

1.8%IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

4.1%IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

6.6%IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

9.2%IOPub message rate exceeded.
T

In [ ]:
model = torchvision.models.resnet18(pretrained=True)#TorchVision

for param in model.parameters():
    param.requires_grad = False
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)
model = model.to(device)

#Hyperparameters
if(ADAM_OPTIMISER):
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
else:
    optimizer = optim.SGD(model.classifier.parameters(), lr=0.001, momentum=0.5)

In [6]:
#Train
best_valid_loss = float('inf')
for epoch in range(EPOCHS):#Range of Epochs
    print(epoch)
    train_loss, train_acc = common.train(model, device, train_iterator, optimizer, criterion)#Train Loss Calculation
    valid_loss, valid_acc = common.evaluate(model, device, valid_iterator, criterion)#Validation Loss Calculation

    if valid_loss < best_valid_loss:#Validation Loss - Is current lower than the saved validation loss.
        best_valid_loss = valid_loss#Save the best loss (lowest)
        torch.save(model.state_dict(), MODEL_SAVE_PATH)#Save the model

    print(f'| Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:05.2f}% | Val. Loss: {valid_loss:.3f} | Val. Acc: {valid_acc*100:05.2f}% |')



| Epoch: 02 | Train Loss: 0.784 | Train Acc: 73.14% | Val. Loss: 0.696 | Val. Acc: 76.04% |
2
| Epoch: 03 | Train Loss: 0.754 | Train Acc: 73.97% | Val. Loss: 0.698 | Val. Acc: 75.68% |
3
| Epoch: 04 | Train Loss: 0.733 | Train Acc: 74.59% | Val. Loss: 0.668 | Val. Acc: 77.15% |
4
| Epoch: 05 | Train Loss: 0.728 | Train Acc: 74.86% | Val. Loss: 0.714 | Val. Acc: 74.78% |
5
| Epoch: 06 | Train Loss: 0.723 | Train Acc: 75.12% | Val. Loss: 0.665 | Val. Acc: 76.55% |
6
| Epoch: 07 | Train Loss: 0.712 | Train Acc: 75.29% | Val. Loss: 0.669 | Val. Acc: 76.81% |
7
| Epoch: 08 | Train Loss: 0.711 | Train Acc: 75.38% | Val. Loss: 0.663 | Val. Acc: 77.27% |
8
| Epoch: 09 | Train Loss: 0.709 | Train Acc: 75.49% | Val. Loss: 0.691 | Val. Acc: 75.52% |
9
| Epoch: 10 | Train Loss: 0.706 | Train Acc: 75.56% | Val. Loss: 0.664 | Val. Acc: 76.59% |
10
| Epoch: 11 | Train Loss: 0.708 | Train Acc: 75.47% | Val. Loss: 0.653 | Val. Acc: 77.83% |
11
| Epoch: 12 | Train Loss: 0.695 | Train Acc: 75.78% | Val.

In [7]:
#3. OUTPUT

model.load_state_dict(torch.load(MODEL_SAVE_PATH)) #Load best weights from file
test_loss, test_acc = common.evaluate(model, device, valid_iterator, criterion) #Test Loss is dependent on
print(f'| Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:05.2f}%')

| Test Loss: 0.621 | Test Acc: 77.93%
